In [ ]:
# Preprocessing

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

There is an abundance of data. Although not all of it might be used, it's still important to inspect each dataset. Let's start with the tournament teams, tournament seeding, and regular season results.

In [ ]:
teams_m = pd.read_csv("/kaggle/input/march-madness-2025/MTeams.csv")

print("Men's tournament teams")
print(teams_m.head())

teams_w = pd.read_csv("/kaggle/input/march-madness-2025/WTeams.csv")

print("\nWomen's tournament teams")
print(teams_w.head())

seeding_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneySeeds.csv")

print("\nMen's tournament seeds")
print(seeding_m.head())

seeding_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneySeeds.csv")

print("\nWomen's tournament seeds")
print(seeding_w.head())

tournament_compact_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyCompactResults.csv")

print("\nMen's Compact Tournament Results")
print(tournament_compact_results_m.head())

tournament_compact_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneyCompactResults.csv")

print("\nWomen's Compact Tournament Results")
print(tournament_compact_results_w.head())

tournament_detailed_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyDetailedResults.csv")

print("\nMen's Detailed Tournament Results")
print(tournament_detailed_results_m.head())

tournament_detailed_results_w = pd.read_csv("/kaggle/input/march-madness-2025//WNCAATourneyDetailedResults.csv")

print("\nWomen's Detailed Tournament Results")
print(tournament_detailed_results_w.head())

regular_season_compact_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MRegularSeasonCompactResults.csv")

print("\nMen's Regular Season Compact Results")
print(regular_season_compact_results_m.head())

regular_season_compact_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WRegularSeasonCompactResults.csv")

print("\nWomen's Regular Season Compact Results")
print(regular_season_compact_results_w.head())

regular_season_detailed_results_m = pd.read_csv("/kaggle/input/march-madness-2025/MRegularSeasonDetailedResults.csv")

print("\nMen's Detailed Season Compact Results")
print(regular_season_detailed_results_m.head())

regular_season_detailed_results_w = pd.read_csv("/kaggle/input/march-madness-2025/WRegularSeasonDetailedResults.csv")

print("\nMen's Detailed Season Compact Results")
print(regular_season_detailed_results_w.head())

conferences_m = pd.read_csv("/kaggle/input/march-madness-2025/MTeamConferences.csv")

print("\nMen's Team Conferences")
print(conferences_m.head())

conferences_w = pd.read_csv("/kaggle/input/march-madness-2025/WTeamConferences.csv")

print("\nWomen's Team Conferences")
print(conferences_w.head())

In [ ]:
def get_seed(df: pd.DataFrame) -> pd.DataFrame:
    df['seed'] = df['Seed'].apply(lambda x: int(x[1:3]))
    df.drop(columns=['Seed'], inplace=True)
    return df

seeding_by_season_m = get_seed(seeding_m)
seeding_by_season_w = get_seed(seeding_w)

print(seeding_by_season_m.head())
print(seeding_by_season_w.head())

In [ ]:
def create_base_submission(df: pd.DataFrame) -> pd.DataFrame:
    # Filter rows where WTeamID < LTeamID and select relevant columns
    base1 = df.loc[df['WTeamID'] < df['LTeamID'], ['Season', 'WTeamID', 'LTeamID']]
    submission1 = base1.assign(
        ID=base1['Season'].astype(str) + "_" + base1['WTeamID'].astype(str) + "_" + base1['LTeamID'].astype(str),
        pred=1
    )
    
    # Filter rows where WTeamID > LTeamID and select relevant columns
    base2 = df.loc[df['WTeamID'] > df['LTeamID'], ['Season', 'WTeamID', 'LTeamID']]
    submission2 = base2.assign(
        ID=base2['Season'].astype(str) + "_" + base2['LTeamID'].astype(str) + "_" + base2['WTeamID'].astype(str),
        pred=0
    )
    
    # Combine and shuffle the submission formats
    submission_combined = pd.concat([submission1, submission2]).sample(frac=1.0, replace=False).reset_index(drop=True)
    
    # Drop unnecessary columns
    return submission_combined.drop(columns=['Season', 'WTeamID', 'LTeamID'])

# Create submissions for men's and women's data
submission_base_m = create_base_submission(tournament_compact_results_m)
submission_base_w = create_base_submission(tournament_compact_results_w)

# Combine submissions
submission_base_combined = pd.concat([submission_base_m, submission_base_w])

# Display the final submission
display(submission_base_combined)

The next steps are to be able to extract the win loss records of each season of each team. We need to the margin of victory, strength of schedule, and simple rating system.

In [ ]:
def margin_of_victory(df: pd.DataFrame) -> pd.DataFrame:
    # First calculate MOV for each game
    df_with_mov = df.copy()
    df_with_mov['MOV'] = df_with_mov['WScore'] - df_with_mov['LScore']

    # Create dataframe for winning teams
    winners = df_with_mov[['Season', 'WTeamID', 'MOV']].rename(columns={'WTeamID': 'TeamID'})
    winners['Win'] = 1
    winners['Loss'] = 0

    # Create dataframe for losing teams (negative MOV)
    losers = df_with_mov[['Season', 'LTeamID', 'MOV']].rename(columns={'LTeamID': 'TeamID'})
    losers['MOV'] = -losers['MOV']
    losers['Win'] = 0
    losers['Loss'] = 1

    # Combine both dataframes
    all_teams = pd.concat([winners, losers])

    # Group by Season and TeamID to get aggregate stats
    team_mov = all_teams.groupby(['Season', 'TeamID']).agg(
        avg_MOV=('MOV', 'mean'),
        total_MOV=('MOV', 'sum'),
        games_played=('MOV', 'count'),
        wins=('Win', 'sum'),
        losses=('Loss', 'sum')
    ).reset_index()

    # Calculate winning percentage
    team_mov['win_pct'] = team_mov['wins'] / team_mov['games_played']

    return team_mov

mov_m = margin_of_victory(regular_season_compact_results_m)
mov_w = margin_of_victory(regular_season_compact_results_w)

print(mov_m.head())
print(mov_w.head())    

In [ ]:
def simple_rating_system(df: pd.DataFrame, mov: pd.DataFrame) -> pd.DataFrame:
    # The mov is passed in from the margin_of_victory function, so avg_MOV is already calculated
    srs = mov.copy()

    # We initialize the SRS to the average MOV
    srs['SRS'] = srs['avg_MOV']

    # We iteratively calculate the SRS until it converges
    for i in range(100):
        # Get opponents' SRS for games where team was winner
        winner_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                             left_on=['Season', 'WTeamID'], 
                             right_on=['Season', 'TeamID'], 
                             how='left')
        winner_sos = winner_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                    left_on=['Season', 'LTeamID'], 
                                    right_on=['Season', 'TeamID'], 
                                    how='left',
                                    suffixes=['_team', '_opp'])
        
        # Get opponents' SRS for games where team was loser
        loser_sos = df.merge(srs[['Season', 'TeamID', 'SRS']], 
                            left_on=['Season', 'LTeamID'], 
                            right_on=['Season', 'TeamID'], 
                            how='left')
        loser_sos = loser_sos.merge(srs[['Season', 'TeamID', 'SRS']], 
                                   left_on=['Season', 'WTeamID'], 
                                   right_on=['Season', 'TeamID'], 
                                   how='left',
                                   suffixes=['_team', '_opp'])
        
        # Combine and calculate average opponent SRS
        winner_opponents = winner_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        loser_opponents = loser_sos[['Season', 'TeamID_team', 'SRS_opp']].rename(
            columns={'TeamID_team': 'TeamID'})
        
        all_opponents = pd.concat([winner_opponents, loser_opponents])
        sos = all_opponents.groupby(['Season', 'TeamID'])['SRS_opp'].mean().reset_index()
        sos.columns = ['Season', 'TeamID', 'SOS']

        # Update SRS
        srs = srs.merge(sos, on=['Season', 'TeamID'], how='left')
        srs['SRS'] = srs['avg_MOV'] + srs['SOS']
        srs.drop(columns=['SOS'], inplace=True)

    return srs

srs_m = simple_rating_system(regular_season_compact_results_m, mov_m)
srs_w = simple_rating_system(regular_season_compact_results_w, mov_w)

print(srs_m.head())
print(srs_w.head())

We will now estimate possessions in order to derive Offensive Rating, Defensive Rating, and Net Rating.

The formula for deriving possessions is Possessions = 0.96 \times (FGA + TO + 0.44 \times FTA - ORB)

In [ ]:
def advanced_ratings(df: pd.DataFrame) -> pd.DataFrame:
    # Estimate possessions
    df['WPossessions'] = 0.96 * (df['WFGA'] + df['WTO'] + 0.44 * df['WFTA'] - df['WOR'])
    df['LPossessions'] = 0.96 * (df['LFGA'] + df['LTO'] + 0.44 * df['LFTA'] - df['LOR'])
    
    # Create dataframes for teams when they win
    w_teams = df[['Season', 'WTeamID', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WPossessions', 'LPossessions']].copy()
    w_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Create dataframes for teams when they lose
    l_teams = df[['Season', 'LTeamID', 'LScore', 'WScore', 'LPossessions', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LPossessions', 'WPossessions']].copy()
    l_teams.columns = ['Season', 'TeamID', 'PointsScored', 'PointsAllowed', 'FGM', 'FGA', 'FGM3', 'FGA3', 'TFM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OwnPossessions', 'OppPossessions']
    
    # Combine both datasets
    all_games = pd.concat([w_teams, l_teams])
    
    # Group by Season and TeamID to get all metrics
    team_ratings = all_games.groupby(['Season', 'TeamID']).agg(
        total_points_scored=('PointsScored', 'sum'),
        total_points_allowed=('PointsAllowed', 'sum'),
        total_field_goals_made=('FGM', 'sum'),
        total_field_goals_attempted=('FGA', 'sum'),
        total_three_point_field_goals_made=('FGM3', 'sum'),
        total_three_point_field_goals_attempted=('FGA3', 'sum'),
        total_free_throws_made=('TFM', 'sum'),
        total_free_throws_attempted=('FTA', 'sum'),
        total_offensive_rebounds=('OR', 'sum'),
        total_defensive_rebounds=('DR', 'sum'),
        total_assists=('Ast', 'sum'),
        total_turnovers=('TO', 'sum'),
        total_steals=('Stl', 'sum'),
        total_blocks=('Blk', 'sum'),
        total_PFs=('PF', 'sum'),
        total_possessions=('OwnPossessions', 'sum'),
        total_opp_possessions=('OppPossessions', 'sum'),
        games_played=('PointsScored', 'count')
    ).reset_index()
    
    # Calculate advanced metrics
    team_ratings['total_points_diff'] = team_ratings['total_points_scored'] - team_ratings['total_points_allowed']
    team_ratings['points_per_game'] = team_ratings['total_points_scored'] / team_ratings['games_played']
    team_ratings['field_goal_percentage'] = 100 * team_ratings['total_field_goals_made'] / team_ratings['total_field_goals_attempted']
    team_ratings['field_goals_made_per_game'] = team_ratings['total_field_goals_made'] / team_ratings['games_played']
    team_ratings['three_point_percentage'] = 100 * team_ratings['total_three_point_field_goals_made'] / team_ratings['total_three_point_field_goals_attempted']
    team_ratings['three_point_field_goals_made_per_game'] = team_ratings['total_three_point_field_goals_made'] / team_ratings['games_played']
    team_ratings['free_throw_percentage'] = 100 * team_ratings['total_free_throws_made'] / team_ratings['total_free_throws_attempted']
    team_ratings['offensive_rebounds_per_game'] = team_ratings['total_offensive_rebounds'] / team_ratings['games_played']
    team_ratings['defensive_rebounds_per_game'] = team_ratings['total_defensive_rebounds'] / team_ratings['games_played']
    team_ratings['rebounds_per_game'] = team_ratings['offensive_rebounds_per_game'] + team_ratings['defensive_rebounds_per_game']
    team_ratings['assists_per_game'] = team_ratings['total_assists'] / team_ratings['games_played']
    team_ratings['turnovers_per_game'] = team_ratings['total_turnovers'] / team_ratings['games_played']
    team_ratings['steals_per_game'] = team_ratings['total_steals'] / team_ratings['games_played']
    team_ratings['blocks_per_game'] = team_ratings['total_blocks'] / team_ratings['games_played']
    team_ratings['personal_fouls_per_game'] = team_ratings['total_PFs'] / team_ratings['games_played']
    team_ratings['offensive_rating'] = 100 * team_ratings['total_points_scored'] / team_ratings['total_possessions']
    team_ratings['defensive_rating'] = 100 * team_ratings['total_points_allowed'] / team_ratings['total_opp_possessions']
    team_ratings['net_rating'] = team_ratings['offensive_rating'] - team_ratings['defensive_rating']
    
    return team_ratings

ratings_m = advanced_ratings(regular_season_detailed_results_m)
ratings_w = advanced_ratings(regular_season_detailed_results_w)

print(ratings_m.head())
print(ratings_w.head())

At this point, we need to start narrowing down the variables of interest.
Here are the ones we will include:
- Season
- TeamID
- ConfAbbrev
- offensive_rating
- defensive_rating
- net_rating
- win_pct
- Seed
- Seed Difference
- rebounds_per_game
- assists_per_game
- steals_per_game
- blocks_per_game
- turnovers_per_game
- personal_fouls_per_game
- field_goal_percentage
- field_goals_made_per_game
- three_point_percentage
- three_point_field_goals_made_per_game
- total_points_scored

In [ ]:
def get_team_conference(df: pd.DataFrame, conferences: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(conferences, on=['TeamID', 'Season'], how='left')
    return df

ratings_m = get_team_conference(ratings_m, conferences_m)
ratings_w = get_team_conference(ratings_w, conferences_w)

def get_seed_difference(df: pd.DataFrame, seeding: pd.DataFrame) -> pd.DataFrame:
    df = df.merge(seeding, on=['TeamID', 'Season'], how='left')
    return df

ratings_m = get_seed(ratings_m, seeding_by_season_m)
ratings_w = get_seed(ratings_w, seeding_by_season_w)

def drop_unnecessary_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns = ['total_points_allowed', 'total_field_goals_made', 'total_field_goals_attempted', 
               'total_three_point_field_goals_made', 'total_three_point_field_goals_attempted', 'total_free_throws_made', 
               'total_free_throws_attempted', 'total_offensive_rebounds', 'total_defensive_rebounds', 'total_assists', 
               'total_turnovers', 'total_steals', 'total_blocks', 'total_PFs', 'total_possessions', 
               'total_opp_possessions', 'games_played']
    df.drop(columns=columns, inplace=True)
    return df

# Drop unnecessary columns
ratings_m = drop_unnecessary_columns(ratings_m)
ratings_w = drop_unnecessary_columns(ratings_w)

print(ratings_m.head())
print(ratings_w.head())

Now that we have the data, we can start to build the model using XGBoost.
The dataframes ratings_m and ratings_w should have all the relevant data we need to build the model.
But the model will need to be able to take in as input all possible matchups between two regular season teams in each season.

In [ ]:
# Import relevant libraries

# xgboost related libraries:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

# sklearn related libraries:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Setting up the data

y_actual_m = pd.read_csv("/kaggle/input/march-madness-2025/MNCAATourneyCompactResults.csv")
y_actual_w = pd.read_csv("/kaggle/input/march-madness-2025/WNCAATourneyCompactResults.csv")

base_data_m = ratings_m.copy()
base_data_w = ratings_w.copy()





y_actual needs to be augmented to include all possible matchups. If a matchup is not in the tourney results, then WScore will be total points scored by the winning team and LScore will be total points scored by the losing team.
This is because the target variable is a float with range [0,1] that predicts the probability of the winning team winning the matchup.
We'll have to use point spread to determine the target variable. Calculations should be based on betting line conventions.
If two teams are in the tournamenet, then the target variable will be a sigmoid curve with a mean of 0.5 and a standard deviation of 0.1. 
The curve will be shifted to the right if the higher seed is the winning team and to the left if the lower seed is the winning team.

In [ ]:
# We will need to reshape the data so that all possible matchups are included in the data. This will be done by creating a new dataframe that includes all possible matchups between two teams in each season.

def create_all_matchups(ratings_df):
    """
    Create all possible matchups between teams for each season,
    ensuring no duplicates (TeamA vs TeamB and TeamB vs TeamA)
    """
    # List to store all matchups
    all_matchups = []
    
    # For each season
    for season in ratings_df['Season'].unique():
        # Get teams for this season
        season_teams = ratings_df[ratings_df['Season'] == season]['TeamID'].tolist()
        
        # Create all possible matchups where TeamID1 < TeamID2 to avoid duplicates
        for i, team1 in enumerate(season_teams):
            for team2 in season_teams[i+1:]:
                all_matchups.append({'Season': season, 'TeamID1': team1, 'TeamID2': team2})
    
    # Convert to DataFrame
    matchups_df = pd.DataFrame(all_matchups)
    
    return matchups_df

# Create all possible matchups for men's and women's teams
matchups_m = create_all_matchups(ratings_m)
matchups_w = create_all_matchups(ratings_w)

print(f"Number of men's matchups: {len(matchups_m)}")
print(f"Number of women's matchups: {len(matchups_w)}")
print(matchups_m.head())


The next step is to fill in the target values for all possible combinations.
The formula involves the game's point spread (if available), the seed difference between 2 teams (if available), the difference in net rating between the 2 teams, and the difference in SRS between the two teams.
The evaluation method uses the Brier Score, which is used by the Kaggle competition to evaluate the model.

In [ ]:
def fill_in_tourney_results(df: pd.DataFrame, tourney_results: pd.DataFrame) -> pd.DataFrame:
    """
    This function isn't the final model. It's just meant to fill in the target variable.
    """

    # Let's merge the matchups with the actual tournament results, we need to preserve all hypothetical matchups
    # First, standardize the tournament results to match our matchup format
    # Create a copy of tournament_results
    tourney_std = tourney_results.copy()
    
    # Ensure WTeamID and LTeamID are integers
    tourney_std['WTeamID'] = tourney_std['WTeamID'].astype(int)
    tourney_std['LTeamID'] = tourney_std['LTeamID'].astype(int)

    # Create columns to match our standardized format where TeamID1 < TeamID2
    tourney_std['TeamID1'] = tourney_std.apply(lambda row: min(row['WTeamID'], row['LTeamID']), axis=1)
    tourney_std['TeamID2'] = tourney_std.apply(lambda row: max(row['WTeamID'], row['LTeamID']), axis=1)
    
    # Add a column to track whether TeamID1 was the winner
    tourney_std['TeamID1_Won'] = tourney_std['TeamID1'] == tourney_std['WTeamID']

    # Add a data source column to identify the source of each result
    tourney_std['DataSource'] = 'Tournament'

    # Select only the columns we need from tourney_std
    tourney_cols = ['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore', 'DataSource']
    tourney_subset = tourney_std[tourney_cols]
    
    # Merge with our matchups dataframe
    result_df = df.merge(tourney_subset, on=['Season', 'TeamID1', 'TeamID2'], how='right')

    return result_df

y_actual_m = tournament_detailed_results_m
y_actual_w = tournament_detailed_results_w

merged_tourney_df_m = fill_in_tourney_results(matchups_m, y_actual_m)
merged_tourney_df_w = fill_in_tourney_results(matchups_w, y_actual_w)

display(merged_tourney_df_m)
display(merged_tourney_df_w)

In [ ]:
def fill_in_regular_results(df: pd.DataFrame, regular_results: pd.DataFrame) -> pd.DataFrame:

    # We need to fill in the regular season matchups where the tournament results are missing
    regular_std = regular_results.copy()

    # Ensure WTeamID and LTeamID are integers
    regular_std['WTeamID'] = regular_std['WTeamID'].astype(int)
    regular_std['LTeamID'] = regular_std['LTeamID'].astype(int)

    # Create columns to match our standardized format where TeamID1 < TeamID2
    regular_std['TeamID1'] = regular_std.apply(lambda row: min(row['WTeamID'], row['LTeamID']), axis=1)
    regular_std['TeamID2'] = regular_std.apply(lambda row: max(row['WTeamID'], row['LTeamID']), axis=1)

    # Add a column to track whether TeamID1 was the winner, astype int to convert boolean to 0 or 1
    regular_std['TeamID1_Won'] = (regular_std['TeamID1'] == regular_std['WTeamID']).astype(int)
    
    # Add a column to identify this as regular season data
    regular_std['DataSource'] = 'RegularSeason'
    
    # Select only the columns we need from regular_std
    regular_cols = ['Season', 'TeamID1', 'TeamID2', 'TeamID1_Won', 'WScore', 'LScore', 'DataSource']
    regular_subset = regular_std[regular_cols]
        
    # Merge with dataframe but only keep rows that exist in regular_subset
    result_df = df.merge(regular_subset, on=['Season', 'TeamID1', 'TeamID2'], how='right')

    return result_df

    # # Fill in the target variable
    # # The target variable is a function of the game's point spread, the seed difference, and the difference in net rating between the two teams
    # # The point spread is the difference in points scored between the winning and losing teams, the bigger the point spread, the more likely the winning team is to win
    # # The seed difference is the difference in seed between the two teams, the bigger the seed difference, the more likely the higher seed is to win
    # # The difference in net rating is the difference in net rating between the two teams, the bigger the difference, the more likely the team with the higher net rating is to win
    # # Target has a range of [0,1], thus a sigmoid function is used to normalize the target variable
    # df['target'] = 1 / (1 + np.exp(-0.1 * (df['MOV'] - 0.1 * df['seed_diff'] - 0.1 * df['net_rating_diff'] - 0.1 * df['srs_diff'])))

    # return df

merged_regular_df_m = fill_in_regular_results(matchups_m, regular_season_detailed_results_m)

display(merged_regular_df_m)

merged_regular_df_w = fill_in_regular_results(matchups_w, regular_season_detailed_results_w)

display(merged_regular_df_w)

In [ ]:
def fill_in_other_results(df: pd.DataFrame, tourney_results: pd.DataFrame, regular_results: pd.DataFrame, ratings: pd.DataFrame, srs: pd.DataFrame, alpha=0.1) -> pd.DataFrame:
    # Filter our the rows in df that already exist in tourney_results and regular_results
    # They key is TeamID1, TeamID2, and Season

    tourney_results_rows = tourney_results[['Season', 'TeamID1', 'TeamID2']].copy()
    regular_results_rows = regular_results[['Season', 'TeamID1', 'TeamID2']].copy()

    existing_results = pd.concat([tourney_results_rows, regular_results_rows])

    # Filter out existing_results from df
    # Merge with indicator to identify rows that exist only in df

    df = df.merge(existing_results, on=['Season', 'TeamID1', 'TeamID2'], how='left', indicator=True)

    # Keep only rows that are in df but not in existing_results
    df = df[df['_merge'] == 'left_only'].drop(columns=['_merge'])

    # For the remaining rows, create columns TeamID1_Won, WScore, LScore, DataSource
    # WScore is the points per game for the winning team for the season
    # LScore is the points per game for the losing team for the season
    # DataSource is 'Other'
    # TeamID1_Won is 1 if TeamID1 has a higher points per game than TeamID2, 0 otherwise

    # Merge with ratings to get points per game for each team
    ratings_subset = ratings[['Season', 'TeamID', 'points_per_game']]

    df = df.merge(ratings_subset, left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'], how='left')

    df.rename(columns={'points_per_game': 'Team1_PPG'}, inplace=True)

    df = df.merge(ratings_subset, left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'], how='left')

    df.rename(columns={'points_per_game': 'Team2_PPG'}, inplace=True)

    # We need to adjust the Team1_PPG and Team2_PPG columns by SRS, which is a measure of team strength
    # The higher the SRS, the stronger the team
    # We will adjust the points per game by the SRS

    srs_subset = srs[['Season', 'TeamID', 'SRS']]
    df = df.merge(srs_subset, left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'], how='left')

    drop_columns = ['TeamID_x', 'TeamID_y', 'TeamID']

    df.drop(columns=drop_columns, inplace=True)

    df = df.merge(srs_subset, left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'], how='left')

    drop_columns = ['TeamID']

    df.drop(columns=drop_columns, inplace=True)

    df.rename(columns={'SRS_x': 'Team1_SRS', 'SRS_y': 'Team2_SRS'}, inplace=True)

    # First, calculate the difference in SRS betweeen the two teams, with a greater difference indicating a stronger team
    df['SRS_diff'] = df['Team1_SRS'] - df['Team2_SRS']
    df['SRS_scaled'] = 2 / (1 + np.exp(-alpha * df['SRS_diff']))
    df['Expected_PPG1'] = df['Team1_PPG'] * df['SRS_scaled']
    df['Expected_PPG2'] = df['Team2_PPG'] * (2 - df['SRS_scaled'])


    # Create TeamID_Won, WScore, LScore, and DataSource columns
    df['TeamID1_Won'] = (df['Expected_PPG1'] > df['Expected_PPG2']).astype(int)
    df['WScore'] = df['Expected_PPG1']
    df['LScore'] = df['Expected_PPG2']
    df['DataSource'] = 'Other'

    # Round the scores to integers
    df['WScore'] = df['WScore'].round().astype(int)
    df['LScore'] = df['LScore'].round().astype(int)

    print(df.columns)

    # Drop unnecessary columns
    df.drop(columns=['Team1_PPG', 'Team2_PPG', 'Team1_SRS', 'Team2_SRS', 'SRS_diff', 'SRS_scaled','Expected_PPG1', 'Expected_PPG2'], inplace=True)

    return df
    
merged_other_df_m = fill_in_other_results(matchups_m, merged_tourney_df_m, merged_regular_df_m, base_data_m, srs_m, alpha=0.006)
display(merged_other_df_m)

# # Show rows where (TeamID1 is in the bottom 10% of SRS and TeamID2 is in the top 10% of SRS) or (TeamID1 is in the top 10% of SRS and TeamID2 is in the bottom 10% of SRS)
# top_bottom_matchups_m = merged_other_df_m[
#     ((merged_other_df_m['Team1_SRS'] < merged_other_df_m['Team1_SRS'].quantile(0.1)) & 
#      (merged_other_df_m['Team2_SRS'] > merged_other_df_m['Team2_SRS'].quantile(0.9))) | 
#     ((merged_other_df_m['Team1_SRS'] > merged_other_df_m['Team1_SRS'].quantile(0.9)) & 
#      (merged_other_df_m['Team2_SRS'] < merged_other_df_m['Team2_SRS'].quantile(0.1)))
# ]

# display(top_bottom_matchups_m)

# # Find the average point spread (absolute value) for these matchups, which is the difference in points per game between the two teams
# avg_point_spread_m = top_bottom_matchups_m['WScore'].sub(top_bottom_matchups_m['LScore']).abs().mean()
# print(f"Average point spread for top 10% vs bottom 10% matchups: {avg_point_spread_m:.2f}")

# Average point spread for top 10% vs bottom 10% matchups: 26.34

merged_other_df_w = fill_in_other_results(matchups_w, merged_tourney_df_w, merged_regular_df_w, base_data_w, srs_w, alpha=0.007)
display(merged_other_df_w)

# # Show rows where (TeamID1 is in the bottom 10% of SRS and TeamID2 is in the top 10% of SRS) or (TeamID1 is in the top 10% of SRS and TeamID2 is in the bottom 10% of SRS)
# top_bottom_matchups_w = merged_other_df_w[
#     ((merged_other_df_w['Team1_SRS'] < merged_other_df_w['Team1_SRS'].quantile(0.1)) & 
#      (merged_other_df_w['Team2_SRS'] > merged_other_df_w['Team2_SRS'].quantile(0.9))) | 
#     ((merged_other_df_w['Team1_SRS'] > merged_other_df_w['Team1_SRS'].quantile(0.9)) & 
#      (merged_other_df_w['Team2_SRS'] < merged_other_df_w['Team2_SRS'].quantile(0.1)))
# ]

# display(top_bottom_matchups_m)

# # Find the average point spread (absolute value) for these matchups, which is the difference in points per game between the two teams
# avg_point_spread_w = top_bottom_matchups_w['WScore'].sub(top_bottom_matchups_w['LScore']).abs().mean()
# print(f"Average point spread for top 10% vs bottom 10% matchups: {avg_point_spread_w:.2f}")

# Average point spread for top 10% vs bottom 10% matchups: 40.19

In [ ]:
def merge_all_results(tourney_df: pd.DataFrame, regular_df: pd.DataFrame, other_df: pd.DataFrame, ratings: pd.DataFrame, srs: pd.DataFrame) -> pd.DataFrame:
    # Note that if a Season, TeamID1, TeamID2 combination exists in multiple dataframes, the order of precedence is tourney_df, regular_df, other_df
    # This means that if a row exists in tourney_df, it will be kept, and the same for regular_df and other_df

    merged_df = tourney_df.merge(regular_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('_tourney', '_regular'))
    
    # Handle precedence - tournament results take priority
    # First, create columns without suffixes using tournament data when available, regular data otherwise
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        merged_df[col] = merged_df[f'{col}_tourney'].combine_first(merged_df[f'{col}_regular'])
    
    # Drop the columns with suffixes as we've consolidated them
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_tourney') or col.endswith('_regular')]
    merged_df = merged_df.drop(columns=cols_to_drop)
    
    # Now merge with other results, again with other having lowest precedence
    merged_df = merged_df.merge(other_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('', '_other'))
    
    # Again handle precedence - merged results take priority over other
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        if f'{col}_other' in merged_df.columns:
            merged_df[col] = merged_df[col].combine_first(merged_df[f'{col}_other'])
    
    # Drop the columns with _other suffix
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_other')]
    merged_df = merged_df.drop(columns=cols_to_drop)

    display(merged_df)

    # Merge with ratings and SRS data for team statistics
    ratings_team1 = ratings.copy()
    for col in ratings_team1.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    # Merge with explicitly renamed columns to avoid suffix issues
    merged_df = merged_df.merge(ratings_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    ratings_team2 = ratings.copy()
    for col in ratings_team2.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(ratings_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    srs_team1 = srs.copy()
    for col in srs_team1.columns:
        if col not in ['Season', 'TeamID']:
            srs_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    srs_team2 = srs.copy()
    for col in srs_team2.columns:
        if col not in ['Season', 'TeamID']:
            srs_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left', suffixes=('', '_srs2'))

    display(merged_df)

    # Scan for duplicate Season, TeamID1, TeamID2, and drop duplicates per the tourney, regular, other precedence
    merged_df = merged_df.drop_duplicates(subset=['Season', 'TeamID1', 'TeamID2'], keep='first')

    print(merged_df.columns)
    
    # Fill missing seeds based on closest SRS value within the same season
    for season in merged_df['Season'].unique():
        season_mask = merged_df['Season'] == season
        
        # Handle team1 seeds
        known_seed_mask1 = merged_df['seed_team1'].notna()
        known_seed_data1 = merged_df[season_mask & known_seed_mask1].copy()
        
        if not known_seed_data1.empty:
            missing_seed_mask1 = merged_df['seed_team1'].isna()
            missing_seed_data1 = merged_df[season_mask & missing_seed_mask1]
            
            for idx, row in missing_seed_data1.iterrows():
                if pd.isna(row['SRS_team1']):
                    # If SRS is also missing, use default value
                    merged_df.loc[idx, 'seed_team1'] = 20
                else:
                    # Find seed of team with closest SRS value
                    known_seed_data1['srs_diff'] = abs(known_seed_data1['SRS_team1'] - row['SRS_team1'])
                    closest_idx = known_seed_data1['srs_diff'].idxmin()
                    merged_df.loc[idx, 'seed_team1'] = merged_df.loc[closest_idx, 'seed_team1']
        else:
            # No teams with seeds in this season, use default
            merged_df.loc[season_mask & merged_df['seed_team1'].isna(), 'seed_team1'] = 20
        
        # Handle team2 seeds using the same approach
        known_seed_mask2 = merged_df['seed_team2'].notna()
        known_seed_data2 = merged_df[season_mask & known_seed_mask2].copy()
        
        if not known_seed_data2.empty:
            missing_seed_mask2 = merged_df['seed_team2'].isna()
            missing_seed_data2 = merged_df[season_mask & missing_seed_mask2]
            
            for idx, row in missing_seed_data2.iterrows():
                if pd.isna(row['SRS_team2']):
                    merged_df.loc[idx, 'seed_team2'] = 20
                else:
                    known_seed_data2['srs_diff'] = abs(known_seed_data2['SRS_team2'] - row['SRS_team2'])
                    closest_idx = known_seed_data2['srs_diff'].idxmin()
                    merged_df.loc[idx, 'seed_team2'] = merged_df.loc[closest_idx, 'seed_team2']
        else:
            merged_df.loc[season_mask & merged_df['seed_team2'].isna(), 'seed_team2'] = 20

    dropped_columns = ['TeamID_x', 'total_points_scored_team1', 'TeamID_y', 'total_points_scored_team2']

    merged_df.drop(columns=dropped_columns, inplace=True)


    # Target variable will be 1 if TeamID1 wins, 0 otherwise
    merged_df.rename(columns={'TeamID1_Won': 'target'}, inplace=True)

    # Move target to the last column
    cols = list(merged_df.columns)
    cols.remove('target')
    merged_df = merged_df[cols + ['target']]

    return merged_df

    
combined_df_m = merge_all_results(merged_tourney_df_m, merged_regular_df_m, merged_other_df_m, ratings_m, srs_m)
display(combined_df_m)

In [ ]:
def merge_all_results_w(tourney_df: pd.DataFrame, regular_df: pd.DataFrame, other_df: pd.DataFrame, ratings: pd.DataFrame, srs: pd.DataFrame) -> pd.DataFrame:
    # Note that if a Season, TeamID1, TeamID2 combination exists in multiple dataframes, the order of precedence is tourney_df, regular_df, other_df
    # This means that if a row exists in tourney_df, it will be kept, and the same for regular_df and other_df

    merged_df = tourney_df.merge(regular_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('_tourney', '_regular'))
    
    # Handle precedence - tournament results take priority
    # First, create columns without suffixes using tournament data when available, regular data otherwise
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        merged_df[col] = merged_df[f'{col}_tourney'].combine_first(merged_df[f'{col}_regular'])
    
    # Drop the columns with suffixes as we've consolidated them
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_tourney') or col.endswith('_regular')]
    merged_df = merged_df.drop(columns=cols_to_drop)
    
    # Now merge with other results, again with other having lowest precedence
    merged_df = merged_df.merge(other_df, on=['Season', 'TeamID1', 'TeamID2'], how='outer', suffixes=('', '_other'))
    
    # Again handle precedence - merged results take priority over other
    for col in ['TeamID1_Won', 'WScore', 'LScore', 'DataSource']:
        if f'{col}_other' in merged_df.columns:
            merged_df[col] = merged_df[col].combine_first(merged_df[f'{col}_other'])
    
    # Drop the columns with _other suffix
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_other')]
    merged_df = merged_df.drop(columns=cols_to_drop)

    display(merged_df)

    # Merge with ratings and SRS data for team statistics
    ratings_team1 = ratings.copy()
    for col in ratings_team1.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    # Merge with explicitly renamed columns to avoid suffix issues
    merged_df = merged_df.merge(ratings_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    ratings_team2 = ratings.copy()
    for col in ratings_team2.columns:
        if col not in ['Season', 'TeamID']:
            ratings_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(ratings_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    srs_team1 = srs.copy()
    for col in srs_team1.columns:
        if col not in ['Season', 'TeamID']:
            srs_team1.rename(columns={col: f"{col}_team1"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team1, left_on=['Season', 'TeamID1'], 
                              right_on=['Season', 'TeamID'], how='left')

    display(merged_df)
    
    srs_team2 = srs.copy()
    for col in srs_team2.columns:
        if col not in ['Season', 'TeamID']:
            srs_team2.rename(columns={col: f"{col}_team2"}, inplace=True)
    
    merged_df = merged_df.merge(srs_team2, left_on=['Season', 'TeamID2'], 
                              right_on=['Season', 'TeamID'], how='left', suffixes=('', '_srs2'))

    display(merged_df)

    # Scan for duplicate Season, TeamID1, TeamID2, and drop duplicates per the tourney, regular, other precedence
    merged_df = merged_df.drop_duplicates(subset=['Season', 'TeamID1', 'TeamID2'], keep='first')

    print(merged_df.columns)
    
    # Fill missing seeds based on closest SRS value within the same season
    for season in merged_df['Season'].unique():
        season_mask = merged_df['Season'] == season
        
        # Handle team1 seeds
        known_seed_mask1 = merged_df['seed_team1'].notna()
        known_seed_data1 = merged_df[season_mask & known_seed_mask1].copy()
        
        if not known_seed_data1.empty:
            missing_seed_mask1 = merged_df['seed_team1'].isna()
            missing_seed_data1 = merged_df[season_mask & missing_seed_mask1]
            
            for idx, row in missing_seed_data1.iterrows():
                if pd.isna(row['SRS_team1']):
                    # If SRS is also missing, use default value
                    merged_df.loc[idx, 'seed_team1'] = 20
                else:
                    # Find seed of team with closest SRS value
                    known_seed_data1['srs_diff'] = abs(known_seed_data1['SRS_team1'] - row['SRS_team1'])
                    closest_idx = known_seed_data1['srs_diff'].idxmin()
                    merged_df.loc[idx, 'seed_team1'] = merged_df.loc[closest_idx, 'seed_team1']
        else:
            # No teams with seeds in this season, use default
            merged_df.loc[season_mask & merged_df['seed_team1'].isna(), 'seed_team1'] = 20
        
        # Handle team2 seeds using the same approach
        known_seed_mask2 = merged_df['seed_team2'].notna()
        known_seed_data2 = merged_df[season_mask & known_seed_mask2].copy()
        
        if not known_seed_data2.empty:
            missing_seed_mask2 = merged_df['seed_team2'].isna()
            missing_seed_data2 = merged_df[season_mask & missing_seed_mask2]
            
            for idx, row in missing_seed_data2.iterrows():
                if pd.isna(row['SRS_team2']):
                    merged_df.loc[idx, 'seed_team2'] = 20
                else:
                    known_seed_data2['srs_diff'] = abs(known_seed_data2['SRS_team2'] - row['SRS_team2'])
                    closest_idx = known_seed_data2['srs_diff'].idxmin()
                    merged_df.loc[idx, 'seed_team2'] = merged_df.loc[closest_idx, 'seed_team2']
        else:
            merged_df.loc[season_mask & merged_df['seed_team2'].isna(), 'seed_team2'] = 20

    dropped_columns = ['TeamID_x', 'total_points_scored_team1', 'TeamID_y', 'total_points_scored_team2']

    merged_df.drop(columns=dropped_columns, inplace=True)


    # Target variable will be 1 if TeamID1 wins, 0 otherwise
    merged_df.rename(columns={'TeamID1_Won': 'target'}, inplace=True)

    # Move target to the last column
    cols = list(merged_df.columns)
    cols.remove('target')
    merged_df = merged_df[cols + ['target']]

    return merged_df

# print(matchups_w.shape)
# print(ratings_w.columns)
# print(srs_w.columns)

# print(merged_tourney_df_w.head())
# print(ratings_w.head())
# print(srs_w.head())

combined_df_w = merge_all_results_w(merged_tourney_df_w, merged_regular_df_w, merged_other_df_w, ratings_w, srs_w)
display(combined_df_w)